In [4]:
import cv2
import numpy as np
import os

def process_image(image_path, output_folder):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return

    # 1. Preprocessing (Convert to Grayscale and Blur)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Adjust kernel size as needed

    # 2. Binarization (Adaptive Thresholding)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY_INV, 11, 2)  # Adjust block size and C

    # 3. Morphological Operations (Closing)
    kernel = np.ones((3, 3), np.uint8)  # Adjust kernel size as needed
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)  # Adjust iterations

    # 4. Segmentation and Contour Extraction
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area (largest first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    mask = np.zeros_like(gray)

    num_contours_to_draw = min(3, len(contours))

    for i in range(num_contours_to_draw):
        cv2.drawContours(mask, contours, i, (255), thickness=cv2.FILLED)

    extracted_shapes = cv2.bitwise_and(image, image, mask=mask)

    output_filename = f"shapes_{os.path.basename(image_path)}"
    output_path = os.path.join(output_folder, output_filename)
    cv2.imwrite(output_path, extracted_shapes)
    print(f"Shapes extracted from {image_path}. Saved as {output_path}")


    background = np.ones_like(image, dtype=np.uint8) * 255
    shapes_on_white = np.where(mask[..., None] == 255, extracted_shapes, background).astype(np.uint8)
    output_filename_white = f"shapes_white_{os.path.basename(image_path)}"
    output_path_white = os.path.join(output_folder, output_filename_white)
    cv2.imwrite(output_path_white, shapes_on_white)



def process_images_in_folder(folder_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            process_image(image_path, output_folder)


# Example usage:
input_folder = "/home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST"  # Replace with your input folder
output_folder = "/home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED"  # Replace with your desired output folder
process_images_in_folder(input_folder, output_folder)

Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/0.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_0.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/1.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_1.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/11.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_11.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/13.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_13.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/14.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_14.png
Shapes extracted from /home/ashwathama/